# L4: Part-of-speech tagging

Part-of-speech tagging is the task of labelling the words (tokens) of a sentence with parts-of-speech such as noun, adjective, and verb. In this lab you will implement the simple, autoregressive fixed-window tagger that was presented in Lecture&nbsp;4.2.

## The data set

The data set for the lab is the English Web Treebank from the [Universal Dependencies Project](http://universaldependencies.org), a corpus containing more than 16,000 sentences (254,000&nbsp;tokens) annotated with, among other things, parts-of-speech. The Universal Dependencies Project distributes its data in the [CoNLL-U format](https://universaldependencies.org/format.html), but for this lab we have converted the data into a simpler format: words and their part-of-speech tags are separated by tabs, sentences are separated by empty lines. The code in the next cell defines a container class for data with this format.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class Dataset():

    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        tmp = []
        with open(self.filename, 'rt', encoding='utf-8') as lines:
            for line in lines:
                line = line.rstrip()
                if line:
                    tmp.append(tuple(line.split('\t')))
                else:
                    yield tmp
                    tmp = []

We load the training data and the development data for this lab:

In [ ]:
train_data = Dataset('/content/drive/MyDrive/TDDE09/labs/l4/train.txt')
dev_data = Dataset('/content/drive/MyDrive/TDDE09/labs/l4/dev.txt')

Both data sets consist of **tagged sentences**. On the Python side of things, a tagged sentence is represented as a list of string pairs, where the first component of each pair represents a word token and the second component represents the word’s tag. The possible tags are listed and exemplified in the [Annotation Guidelines](http://universaldependencies.org/u/pos/all.html) of the Universal Dependencies Project. Run the next code cell to see an example of a tagged sentence.

In [ ]:
print(list(train_data)[42])
print(list(dev_data)[42])


[('There', 'PRON'), ('has', 'AUX'), ('been', 'VERB'), ('talk', 'NOUN'), ('that', 'SCONJ'), ('the', 'DET'), ('night', 'NOUN'), ('curfew', 'NOUN'), ('might', 'AUX'), ('be', 'AUX'), ('implemented', 'VERB'), ('again', 'ADV'), ('.', 'PUNCT')]
[('We', 'PRON'), ("'ve", 'AUX'), ('grown', 'VERB'), ('up', 'ADP'), ('.', 'PUNCT')]


## Tagger interface

The tagger that you will implement in this lab follows a simple interface with just one method:

In [ ]:
class Tagger(object):

    def predict(self, sentence):
        raise NotImplementedError

The single method of this interface has the following specification:

**predict** (*self*, *sentence*)

> Returns the list of predicted tags (a list of strings) for a single *sentence* (a list of string tokens).

One trivial implementation of this interface is a tagger that always predicts the same tag for every word, independently of the input:

In [ ]:
class ConstantTagger(Tagger):
    
    def __init__(self, the_tag):
        self.the_tag = the_tag
    
    def predict(self, words):
        return [self.the_tag] * len(words)

## Problem 1: Implement an evaluation function

Your first task is to implement a function that computes the accuracy of a tagger on gold-standard data.

In [ ]:
def accuracy(tagger, gold_data):
    # TODO: Replace the next line with your own code
    tot = 0
    correct = 0

    for words in gold_data:
        tot += len(words)
        gold_words = [i[1] for i in words]
        g_words = [i[0] for i in words]

        pred_words = tagger.predict(g_words)
        
        correct += sum(1 for x,y in zip(gold_words,pred_words) if x == y)
    
    return correct / tot

In [ ]:
const_tagger = ConstantTagger('NOUN')

acc = accuracy(const_tagger, dev_data)
acc

Streaming output truncated to the last 5000 lines.
['ADJ', 'NOUN', 'PUNCT']
['NOUN', 'NOUN', 'NOUN']

['PUNCT']
['NOUN']

['SYM']
['NOUN']

['PROPN', 'PROPN', 'ADJ', 'NOUN', 'PUNCT', 'NUM', 'ADJ', 'NOUN', 'PUNCT', 'NUM', 'NOUN', 'PUNCT', 'X']
['NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN']

['PUNCT']
['NOUN']

['PUNCT', 'NOUN']
['NOUN', 'NOUN']

['PROPN', 'PROPN']
['NOUN', 'NOUN']

['NUM', 'NUM', 'NOUN']
['NOUN', 'NOUN', 'NOUN']

['PRON', 'VERB', 'ADP', 'PROPN', 'PROPN', 'ADP', 'PROPN', 'VERB', 'PRON', 'NOUN', 'PUNCT']
['NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN']

['PROPN', 'VERB', 'PROPN', 'ADJ', 'VERB', 'NUM', 'NOUN', 'PRON', 'DET', 'VERB', 'ADV', 'ADP', 'PROPN', 'PUNCT']
['NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN']

['ADV', 'PUNCT', 'PRON', 'ADV', 'VERB', 'PRON', 'AUX', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'PRON', 'AUX', '

0.1668919993637665

Your implementation should conform to the following specification:

**accuracy** (*tagger*, *gold_data*)

> Computes the accuracy of the *tagger* on the gold-standard data *gold_data* (an iterable of tagged sentences) and returns it as a float. Recall that the accuracy is defined as the percentage of tokens to which the tagger assigns the correct tag (as per the gold standard).

### 🤞 Test your code

Test your code by computing the accuracy on the development set of a trivial tagger that tags each word as a noun. The expected value is 16.69%.

## Problem 2: Implement a baseline

Before you start working on the tagger as such, we ask you to first implement a simple baseline:

> Tag each input word with the most frequent tag for that word in the training data. If an input word does not occur in the training data, tag it with the overall most frequent tag in the training data. Break ties by choosing that tag which comes first in the alphabetical order.

To implement the baseline, you need to implement both a class `BaselineTagger` and a function `train_baseline`. A `BaselineTagger` has two fields: a dictionary mapping each word in the training data to the most frequent tag for that word, and a string representing the fallback tag (overall most frequent tag in the training data). Both of these fields are set in the `train_baseline` function.

In [ ]:
class BaselineTagger(Tagger):

    def __init__(self):
        self.most_frequent = {}
        self.fallback = None

    def predict(self, words):
        # TODO: Replace the next line with your own code
        res = []
        for word, tag in words:
            if word in self.most_frequent:
                res.append(self.most_frequent[word])
            else:
                res.append(self.fallback)
        #print(words)
        #print(res)
        return res


def train_baseline(train_data):
    # TODO: Replace the next line with your own code

    ret = BaselineTagger()

    temp = {}
    fallback = {}

    all_words = set()

    for sent in train_data:
        for word_and_tag in sent:
            if word_and_tag in temp:
                temp[word_and_tag] += 1
            else:
                temp[word_and_tag] = 1
            
            if word_and_tag[1] in fallback:
                fallback[word_and_tag[1]] += 1
            else:
                fallback[word_and_tag[1]] = 1

            all_words.add(word_and_tag[0])


    most_frequent = {}
    for word in all_words:
        best_tag = ''
        freq = 0
        for word_and_tag, occ in temp.items():
            if word == word_and_tag[0]:
                if occ > freq:
                    best_tag = word_and_tag[1]
                    freq = occ
        most_frequent[word] = best_tag

    
    #print(most_frequent)
    ret.most_frequent = most_frequent
    ret.fallback = max(fallback, key=fallback.get)
    return ret

In [ ]:
baseline_tagger = train_baseline(train_data)

In [ ]:
acc = accuracy(baseline_tagger, dev_data)
acc

0.8564100524892636

### 🤞 Test your code

Test your implementation by computing the accuracy of the baseline tagger on the development data. The expected value is 85.61%.

## Problem 3: Create the vocabularies

As in previous labs, you will need an explicit representation of your vocabulary. Here we actually have two vocabularies: one for the words and one for the tags. Both should be represented as dictionaries that map words/tags to a contiguous range of integers, starting at zero.

The next cell contains skeleton code for a function `make_vocabs` that constructs the two vocabularies from gold-standard data. The code cell also defines a name for the ‘unknown word’ (`UNK`) and for an additional pseudoword that you will use as a placeholder for undefined values (`PAD`).

In [ ]:
PAD = '<pad>'
UNK = '<unk>'

def make_vocabs(gold_data):
    #print(list(gold_data))

    words = {}
    tags = {}

    curr_word_idx = 2
    curr_tag_idx = 1

    words[PAD] = 0
    words[UNK] = 1

    tags[PAD] = 0

    for sent in list(gold_data):
        for word, tag in sent:
            if word not in words:
                words[word] = curr_word_idx
                curr_word_idx += 1
            
            if tag not in tags:
                tags[tag] = curr_tag_idx
                curr_tag_idx += 1

    return words, tags

In [ ]:
words, tags = make_vocabs(train_data)

print(len(words))
print(len(tags))

19674
18


Complete the code according to the following specification:

**make_vocabs** (*gold_data*)

> Returns a pair of dictionaries mapping the unique words and tags in the gold-standard data *gold_data* (an iterable over tagged sentences) to contiguous ranges of integers starting at zero. The word dictionary contains the pseudowords `PAD` (index&nbsp;0) and `UNK` (index&nbsp;1); the tag dictionary contains `PAD` (index&nbsp;0).

### 🤞 Test your code

Test your implementation by computing the total number of unique words and tags in the training data (including the pseudowords). The expected values are 19,674&nbsp;words and 18&nbsp;tags.

## Problem 4: Fixed-window tagger

Your main task in this lab is to implement a complete, autoregressive part-of-speech tagger based on the fixed-window architecture. This implementation has four parts: the fixed-window model; a tagger that uses the fixed-window model to make predictions; a function that generates training examples for the tagger; and the training function.

**⚠️ We expect that solving this problem will take you the longest time in this lab.**

### Problem 4.1: Implement the fixed-window model

The architecture of the fixed-window model is presented in Lecture&nbsp;4.2. An input to the network takes the form of a $k$-dimensional vector of word ids and/or tag ids. Each integer $i$ is mapped to an $e_i$-dimensional embedding vector. These vectors are concatenated to form a vector of length $e_1 + \cdots + e_k$, and sent through a feed-forward network with a single hidden layer and a rectified linear unit (ReLU).

#### Default features

We ask you to implement a fixed-window model with the following features ($k=4$):

0. current word
1. previous word
2. next word
3. tag predicted for the previous word

Whenever the value of a feature is undefined, you should use the special value `PAD`.

#### Embedding specifications

To make your implementation of the fixed-window model useful for a range of different applications (including the parser that you will build in lab&nbsp;5), it should support other feature sets than the default model. To this end, the constructor of your model should accept a list of what we call *embedding specifications*. An embedding specification is a triple $(m, n, e)$ consisting of three integers. Such a triple specifies that the model should include $m$ instances of an embedding from $n$ items to vectors of size $e$. All of the $m$ instances are to share their weights. In this lab, the embeddings will be embeddings for words and tags. For example, to instantiate the default feature model, you would initialise the model with the following specifications:

``
[(3, num_words, word_dim), (1, num_tags, tag_dim)]
``

This specifies that the model should use 3 instances of an embedding from *num_words* words to vectors of length *word_dim*, and 1 instance of an embedding from *num_tags* tags to vectors of length *tag_dim*. All 3 instances of the word embedding would share their weights. If you rather wanted to have word embeddings with separate weights, you would initialise the model with the following specifications:

``
[(1, num_words, word_dim), (1, num_words, word_dim), (1, num_words, word_dim), (1, num_tags, tag_dim)]
``

We recommend that you initialize the weights of each embedding with values drawn from $\mathcal{N}(0, 10^{-2})$.

#### Hyperparameters

The network architecture introduces a number of hyperparameters. The following choices are reasonable defaults:

* width of each word embedding: 50
* width of each tag embedding: 10
* size of the hidden layer: 100

The next cell contains skeleton code for the implementation of the fixed-window model.

In [ ]:
import torch.nn as nn
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class FixedWindowModel(nn.Module):

    def __init__(self, embedding_specs, hidden_dim, output_dim):
        super().__init__()
        # TODO: Add your code here
        
        # Random
        self.embeddings = nn.ModuleList((nn.Embedding(i[1], i[2])) for i in embedding_specs for j in range(i[0]))  

        glove = torch.load('/content/drive/MyDrive/TDDE09/labs/l4/glove.pt')
        # count = 0
        for embed in self.embeddings:
          embed.from_pretrained(glove, freeze = False)
          # count += 1
        
        self.linear1   = nn.Linear(50*3 + 10, hidden_dim)
        self.relu      = nn.ReLU()
        self.linear2   = nn.Linear(hidden_dim, output_dim)
        
    # Features is B x k where B == batch_size and k is total embeddings (k = 4, 3 word embs and 1 tag emb)
    def forward(self, features):
        # TODO: Replace the next line with your own code
        #print(self.embeddings)
        #print("feature size: ", features.size())

        rows=features.size()[0]
        cols=features.size()[1]
        feat_v = torch.empty(features.size()[0], 160)
        for row in range(0,rows):
            target_embedding = self.embeddings[0].weight[features[row][0]]
            for column in range(1,cols):
                word_embedding = self.embeddings[column].weight[features[row][column]]
                target_embedding = torch.cat((target_embedding, word_embedding),-1)
                #print("target_embedding size: ", target_embedding.size())
            feat_v[row] = target_embedding
        
        linear1 = self.linear1(feat_v)
        rel     = self.relu(linear1)
        output  = self.linear2(rel)   

        return output

In [ ]:
test = FixedWindowModel([(3, len(words), 50), (1, len(tags), 10)], 100, 18)

Your implementation should meet the following specification:

**__init__** (*self*, *embedding_specs*, *hidden_dim*, *output_dim*)

> A fixed-window model is initialized with a list of specifications for the embeddings the network should use (*embedding_specs*), the size of the hidden layer (*hidden_dim*), and the size of the output layer (*output_dim*).

**forward** (*self*, *features*)

> Computes the network output for a given feature representation *features*. This is a tensor of shape $B \times k$ where $B$ is the batch size (number of samples in the batch) and $k$ is the total number of embeddings specified upon initialisation. For example, for the default feature model, $k=4$, as this model includes 3 (weight-sharing) word embeddings and 1 tag embedding.

#### 💡 Hint on the implementation

You will have to construct embeddings based on the embedding specifications. It is natural to store these embeddings in a list- or dictionary-valued attribute of the `FixedWindowModel` object. However, in order to expose the embeddings to the auto-differentiation magic of PyTorch (so that their weights are updated during training), you must instead store them in an [`nn.ModuleList`](https://pytorch.org/docs/stable/nn.html#torch.nn.ModuleList) or [`nn.ModuleDict`](https://pytorch.org/docs/stable/nn.html#torch.nn.ModuleDict).

### Problem 4.2: Implement the tagger

The next step is to implement the tagger itself. The tagger will use the simple algorithm that was presented in Lecture&nbsp;4.2: It processes an input sentence from left to right, and at each position, predicts the tag for the current word based on the features extracted from the current feature window.

In [ ]:
class FixedWindowTagger(Tagger):

    def __init__(self, vocab_words, vocab_tags, word_dim=50, tag_dim=10, hidden_dim=100):
        embedding_specs = [(3, len(vocab_words), word_dim), (1, len(vocab_tags), tag_dim)]

        self.vocab_words = vocab_words
        self.vocab_tags  = vocab_tags
        
        self.model = FixedWindowModel(embedding_specs, hidden_dim, len(vocab_tags))

        # TODO: Replace the next line with your own code
        pass

    def featurize(self, words, i, pred_tags):
        # TODO: Replace the next line with your own code
        # All cases
      if len(words) == 1: # If only one word is sent in
            return torch.LongTensor([words[i], 0, 0, 0])
      elif i == len(words) - 1: #If current word end of list
            return torch.LongTensor([words[i], words[i - 1], 0, pred_tags[i - 1]])
      elif i == 0: #If current word beginning of list
            return torch.LongTensor([words[i], 0, words[i + 1], 0])
      else:  # If middle of words
            return torch.LongTensor([words[i], words[i - 1], words[i + 1], pred_tags[i - 1]])

    def predict(self, words):
        word_ids = []
        for word in words:
              if word in self.vocab_words:
                    word_ids.append(self.vocab_words[word])
              else:
                    word_ids.append(0)

        predicted_tags = []
        for i in range(len(word_ids)):
              curr_feature = self.featurize(word_ids, i, predicted_tags)
              res = self.model.forward(curr_feature.view(1,4))
              not_used, tag = torch.max(res.squeeze(0), 0)
              predicted_tags.append(tag)
        
        # Convert tag ids to strings
        predicted_tag_strings = []
        for pred_tag_id in predicted_tags:
          for key, val in self.vocab_tags.items():
                if val == pred_tag_id:
                      predicted_tag_strings.append(key)
                      
        return predicted_tag_strings

In [ ]:
tagger = FixedWindowTagger(words, tags)

print(tagger.predict(["the","cat", "sat","on","the","floor"])) # gives weird results

# print(accuracy(tagger,dev_data))

FileNotFoundError: ignored

Complete the skeleton code by implementing the methods of this interface:

**__init__** (*self*, *vocab_words*, *vocab_tags*, *word_dim* = 50, *tag_dim* = 10, *hidden_dim* = 100)

> Creates a new fixed-window model of appropriate dimensions and sets up any other data structures that you consider relevant. The parameters *vocab_words* and *vocab_tags* are the word vocabulary and tag vocabulary. The parameters *word_dim* and *tag_dim* specify the embedding width for the word embeddings and tag embeddings.

**featurize** (*self*, *words*, *i*, *pred_tags*)

> Extracts features from the specified tagger configuration according to the default feature model. The configuration is specified in terms of the words in the input sentence (*words*, a list of word ids), the position of the current word (*i*), and the list of already predicted tags (*pred_tags*, a list of tag ids). Returns a tensor that can be fed to the fixed-window model.

**predict** (*self*, *words*)

> Processes the input sentence *words* (a list of string tokens) and makes calls to the fixed-window model to predict the tag of each word. Returns the list of the predicted tags (strings).

### Problem 4.3: Generate the training examples

Your next task is to implement a function that generates the training examples for the tagger. You will train the tagger as usual, using minibatch training.

In [ ]:
def training_examples(vocab_words, vocab_tags, gold_data, tagger, batch_size=100):
    batch_x, batch_y = [], []

    for sentence in gold_data:
        word_ids = []
        tag_ids = []
        for word, tag in sentence:
            word_ids.append(vocab_words[word])
            tag_ids.append(vocab_tags[tag])

        for idx, tag in enumerate(tag_ids):
            feature = tagger.featurize(word_ids, idx, tag_ids)
            batch_x.append(feature)
            batch_y.append(tag)

            # If batch contains batch size examples, yield them and empty x and y.
            if len(batch_x) == batch_size:
                yield torch.stack(batch_x, dim=0), torch.LongTensor(batch_y)
                batch_x, batch_y = [], []
    

tagger = FixedWindowTagger(words, tags)
for x, y in training_examples(words, tags, list(train_data), tagger):
  break

Your code should comply with the following specification:

**training_examples** (*vocab_words*, *vocab_tags*, *gold_data*, *tagger*, *batch_size* = 100)

> Iterates through the given *gold_data* (an iterable of tagged sentences), encodes it into word ids and tag ids using the specified vocabularies *vocab_words* and *vocab_tags*, and then yields batches of training examples for gradient-based training. Each batch contains *batch_size* examples, except for the last batch, which may contain fewer examples. Each example in the batch is created by a call to the `featurize` function of the *tagger*.

### Problem 4.4: Training loop

What remains to be done is the implementation of the training loop. This should be a straightforward generalization of the training loops that you have seen so far. Complete the skeleton code in the cell below:

In [ ]:
import torch.optim as optim
import torch.nn.functional as F

def train_fixed_window(train_data, n_epochs=1, batch_size=100, lr=1e-2):
    tagger = FixedWindowTagger(words, tags)
    vocab_words, vocab_tags = make_vocabs(train_data)
    model = tagger.model
  
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses = []
  
    for epoch in range(n_epochs): 
        count = 1
        progress = 0
        train_loss = 0
        avg_loss = 0
        model.train()
        for x, y in training_examples(vocab_words, vocab_tags,train_data, tagger):
            optimizer.zero_grad()

            #Forward pass
            output = model.forward(x) 

            #Compute the loss
            loss = F.cross_entropy(output,  y) 
            train_loss += loss.item()
            progress += len(y)
            
            #clear_output()
            print("Running Train loss: " + str(train_loss/count))

            count += 1
            
            #Backward pass
            loss.backward()
            
            #Update the parameter of the model
            optimizer.step()

    return tagger

Here is the specification of the training function:

**train_fixed_window** (*train_data*, *n_epochs* = 1, *batch_size* = 100, *lr* = 1e-2)

> Trains a fixed-window tagger from a set of training data *train_data* (an iterable over tagged sentences) using minibatch gradient descent and returns it. The parameters *n_epochs* and *batch_size* specify the number of training epochs and the minibatch size, respectively. Training uses the cross-entropy loss function and the [Adam optimizer](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) with learning rate *lr*.

The next code cell trains a tagger and evaluates it on the development data:

In [ ]:
tagger_windowmodel = train_fixed_window(train_data)

Running Train loss: 2.9193897247314453
Running Train loss: 2.7485557794570923
Running Train loss: 2.598641792933146
Running Train loss: 2.423242598772049
Running Train loss: 2.28271005153656
Running Train loss: 2.2415968775749207
Running Train loss: 2.206325445856367
Running Train loss: 2.1823378652334213
Running Train loss: 2.1541612810558743
Running Train loss: 2.1215951442718506
Running Train loss: 2.1161048195578833
Running Train loss: 2.1074467301368713
Running Train loss: 2.095319922153766
Running Train loss: 2.074682687010084
Running Train loss: 2.043636918067932
Running Train loss: 2.0263777300715446
Running Train loss: 2.0046615881078385
Running Train loss: 1.9769301414489746
Running Train loss: 1.9607809531061273
Running Train loss: 1.9417296826839447
Running Train loss: 1.9157008500326247
Running Train loss: 1.8772994063117288
Running Train loss: 1.8586324246033379
Running Train loss: 1.95438248415788
Running Train loss: 1.928841414451599
Running Train loss: 1.90628784436445

In [ ]:
print('{:.4f}'.format(accuracy(tagger_windowmodel, dev_data)))

0.8399


In [ ]:
class Dataset():

    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        with open(self.filename, 'rt', encoding='utf-8') as lines:
            tmp = []
            for line in lines:
                if not line.startswith('#'):  # Skip lines with comments
                    line = line.rstrip()
                    if line:
                        columns = line.split('\t')
                        if columns[0].isdigit():  # Skip range tokens
                            tmp.append(columns)
                    else:
                        yield tmp
                        tmp = []

with open('en_ewt-ud-dev-retagged.conllu', 'wt') as target:
    for sentence in Dataset('en_ewt-ud-dev.conllu'):
        words = [columns[1] for columns in sentence]
        for i, t in enumerate(tagger.predict(words)):
            sentence[i][3] = t
        for columns in sentence:
            print('\t'.join(c for c in columns), file=target)
        print(file=target)

# (columns[1], columns[3], int(columns[6])))

FileNotFoundError: ignored

**⚠️ Your submitted notebook must contain output demonstrating at least 88% accuracy on the development set.**

## Problem 5: Pre-trained embeddings (reflection)

Many neural systems for natural language processing use pre-trained word embeddings, either to augment or to replace randomly initialised task-based embeddings. In this problem, you will investigate whether pre-trained embeddings help your part-of-speech tagger.

The file `glove.pt` contains a PyTorch tensor containing 50-dimensional pre-trained word embeddings from the [GloVe project](https://nlp.stanford.edu/projects/glove/). You can load this tensor using the command

```
glove = torch.load('glove.pt')
```

and should be able to use it as a drop-in replacement for your randomly initialized word embeddings, assuming that the words in your vocabulary are numbered in the order in which they are found in the training data. Have a look at the documentation of the class [`nn.Embedding`](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) to learn how to do this.

Run experiments to assess the effect that pre-trained embeddings have on (a)&nbsp;the accuracy of the tagger, and (b)&nbsp;the speed of learning, i.e., the number of training examples it takes to reach a certain loss. Document your exploration in a short reflection piece (ca. 150&nbsp;words). Respond to the following prompts:

* How did you integrate the pre-trained embeddings into your system? What did you measure? What results did you get?

We integrated the pretraining by instantiating the embeddings with the glove torch tensor. This was done in the TaggerModel. We meassured the accuracy of the pretrained against the original tagger. The pre-trained model reached a much lower running-loss of 0.3539 while the randomly instantiated embeddings reached a running loss of 0.54 after 1 epoch.

* Based on what you know about word embeddings and transfer learning, did you expect your results? How do you explain them?

Yes, we expected the running loss to decrease much faster with the pretrained embeddings. This is because transfer learning makes the model focus more on the learning of the linear and relu layers instead of learning the embedding layers.

* What did you learn? How, exactly, did you learn it? Why does this learning matter?

In this lab we learned about baselines. We also found it intresting to see the pre-trained embeddings and how they work by decreasing the learning time. 

*TODO: Insert your report here*

**🥳 Congratulations on finishing this lab! 🥳**